<a href="https://colab.research.google.com/github/chhyyi/aiffel/blob/main/LMS/aiffel_exp10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# exp10 : 번역기 만들기
그동안의 경험을 바탕으로 일단 tensorflow, keras 버전을 lms와 맞추고 시작하겠습니다.
___런타임 재시작 필요___

In [ ]:
!pip3 uninstall tensorflow -y
!pip3 uninstall keras -y
!pip3 install tensorflow==2.6.0
!pip3 install keras==2.6.0

In [ ]:
!pip3 install pandas==1.3.3
!pip3 install numpy==1.21.4

In [ ]:
file_path = r"/content/drive/MyDrive/colabdata/modulabs/lms_exp10/fra.txt"

# step(1) 정제, 정규화, 전처리
## 데이터 불러오기
33,000 정도의 샘플만 불러오라고 하여 그렇게 합니다.

In [ ]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np

In [ ]:
import os
lines = pd.read_csv(file_path, names=['eng', 'fra', 'cc'], sep='\t')
print('전체 샘플의 수 :',len(lines))
lines.sample(5) #샘플 5개 출력

전체 샘플의 수 : 194513


,eng,fra,cc
60085,What a beautiful scene!,Quelle belle scène !,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
182348,She used Skype frequently because she was over...,"Comme elle était à l'étranger, elle téléphonai...",CC-BY 2.0 (France) Attribution: tatoeba.org #8...
53009,What did it feel like?,Qu'avez-vous ressenti ?,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
31732,The tree fell down.,L'arbre se coucha.,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
80521,Look at that red building.,Regardez cet immeuble rouge.,CC-BY 2.0 (France) Attribution: tatoeba.org #6...


In [ ]:
lines = lines[['eng', 'fra']][:33000] # 33000개 샘플 사용
lines.sample(5)

,eng,fra
9675,Come back soon.,Ne sois pas long.
11314,Let me hear it.,Laissez-moi l'entendre.
4998,I must hurry.,Je dois me hâter.
2883,Don't shoot.,Ne tirez pas !
7876,Is Tom around?,Tom est-il dans les parages ?


구두점 (punctuation) 분리하기
exp6 작사가 만들기에서 많이 참고하였습니다.

In [ ]:
import re
lines['eng_']=lines['eng'].str.replace(r"([?.!,¿])", r" \1 ",regex=True)
lines['fra_']=lines['fra'].str.replace(r"([?.!,¿])", r" \1 ",regex=True)

lines.sample(5)

,eng,fra,eng_,fra_
9507,You're vulgar.,Tu es vulgaire.,You're vulgar .,Tu es vulgaire .
3492,Is Tom here?,Est-ce que Tom est ici ?,Is Tom here ?,Est-ce que Tom est ici ?
32804,We were both tired.,Nous étions toutes les deux fatiguées.,We were both tired .,Nous étions toutes les deux fatiguées .
27224,Where are my keys?,Où sont mes clés ?,Where are my keys ?,Où sont mes clés ?
28337,Do you like school?,Aimez-vous l'école ?,Do you like school ?,Aimez-vous l'école ?


In [ ]:
lines.drop(['eng', 'fra'], axis=1, inplace=True)
lines.sample(5)

,eng_,fra_
14853,Isn't it lovely ?,N'est-ce pas charmant ?
20354,That's upsetting .,C'est déplaisant .
19996,Please sit still .,"S'il vous plaît , tenez-vous tranquille ."
5667,Put it there .,Mettez-le là .
5568,Let Tom stay .,Laisse Tom rester .


## 소문자로 바꾸기

In [ ]:
#for i in len(lines):
lines_lower=pd.DataFrame()
lines_lower['eng']=lines.apply(lambda x:str(x['eng_']).lower().strip(), axis=1)
lines_lower['fra']=lines.apply(lambda x:str(x['fra_']).lower().strip(), axis=1)
lines_lower

,eng,fra
0,go .,va !
1,go .,marche .
2,go .,bouge !
3,hi .,salut !
4,hi .,salut .
...,...,...
32995,what did tom steal ?,qu'est-ce que tom a volé ?
32996,what did tom write ?,qu'est-ce que tom a écrit ?
32997,what did they want ?,que voulaient-ils ?
32998,what did they want ?,que voulaient-elles ?


In [89]:
eng=lines_lower.apply(lambda x:str(x['eng']).split(), axis=1)
fra=lines_lower.apply(lambda x:str(x['fra']).split(), axis=1)
#print(eng)
#print(fra)
print(type(eng), type(eng[0]))

<class 'pandas.core.series.Series'> <class 'list'>


## step(2) decoder용 데이터에 시작, 종료 토큰 넣기

In [90]:
fra = fra.apply(lambda x : ['\t']+ x + ['\n'])
print(fra.head())

0        [\t, va, !, \n]
1    [\t, marche, ., \n]
2     [\t, bouge, !, \n]
3     [\t, salut, !, \n]
4     [\t, salut, ., \n]
dtype: object


In [91]:
print(type(fra), type(fra[0]))

<class 'pandas.core.series.Series'> <class 'list'>


In [92]:
eng_tokenizer = Tokenizer()   # 문자 단위로 Tokenizer를 생성합니다. 
eng_tokenizer.fit_on_texts(eng)               # 50000개의 행을 가진 eng의 각 행에 토큰화를 수행
input_text = eng_tokenizer.texts_to_sequences(eng)    # 단어를 숫자값 인덱스로 변환하여 저장
input_text[:5]
print(type(input_text),type(input_text[0]))

<class 'list'> <class 'list'>


## step(3) tokenizer 생성, encoder input/output 분리

In [93]:
fra_tokenizer = Tokenizer()   # 문자 단위로 Tokenizer를 생성합니다. 
fra_tokenizer.fit_on_texts(fra)                 # fra의 각 행에 토큰화를 수행
target_text = fra_tokenizer.texts_to_sequences(fra)     # 단어를 숫자값 인덱스로 변환하여 저장

eng_vocab_size = len(eng_tokenizer.word_index) + 1
fra_vocab_size = len(fra_tokenizer.word_index) + 1
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)

max_eng_seq_len = max([len(line) for line in input_text])
max_fra_seq_len = max([len(line) for line in target_text])
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

print('전체 샘플의 수 :',len(lines))

영어 단어장의 크기 : 4810
프랑스어 단어장의 크기 : 9989
영어 시퀀스의 최대 길이 8
프랑스어 시퀀스의 최대 길이 15
전체 샘플의 수 : 33000


In [97]:
encoder_input=input_text

In [98]:
decoder_input=[x[1:] for x in target_text]

In [99]:
decoder_output=[x[:-1] for x in target_text]

In [100]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model

In [101]:
# 입력 텐서 생성.
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(eng_vocab_size,128)(encoder_inputs)
# hidden size가 256인 인코더의 LSTM 셀 생성
encoder_lstm = LSTM(units = 256, return_state = True)
# 디코더로 전달할 hidden state, cell state를 리턴. encoder_outputs은 여기서는 불필요.
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# hidden state와 cell state를 다음 time step으로 전달하기 위해서 별도 저장.
encoder_states = [state_h, state_c]

In [102]:
# 입력 텐서 생성.
decoder_inputs = Input(shape=(None,))
fra_emb =  Embedding(fra_vocab_size,256)(decoder_inputs)
# hidden size가 256인 인코더의 LSTM 셀 생성
decoder_lstm = LSTM(units = 256, return_sequences = True, return_state=True)
# decoder_outputs는 모든 time step의 hidden state
decoder_outputs, _, _= decoder_lstm(fra_emb, initial_state = encoder_states)

In [103]:
decoder_softmax_layer = Dense(fra_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

In [104]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_21 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
input_22 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_18 (Embedding)        (None, None, 128)    615680      input_21[0][0]                   
__________________________________________________________________________________________________
embedding_19 (Embedding)        (None, None, 256)    2557184     input_22[0][0]                   
____________________________________________________________________________________________

In [105]:
encoder_input = pad_sequences(encoder_input, maxlen = max_eng_seq_len, padding='post')
print(encoder_input)
decoder_input = pad_sequences(decoder_input, maxlen = max_fra_seq_len, padding='post')
print(decoder_input)
decoder_target = pad_sequences(decoder_output, maxlen = max_fra_seq_len, padding='post')
print(decoder_target)

[[26  1  0 ...  0  0  0]
 [26  1  0 ...  0  0  0]
 [26  1  0 ...  0  0  0]
 ...
 [41 34 29 ...  0  0  0]
 [41 34 29 ...  0  0  0]
 [41 34  4 ...  0  0  0]]
[[  63    7    2 ...    0    0    0]
 [ 360    3    2 ...    0    0    0]
 [ 710    7    2 ...    0    0    0]
 ...
 [  27 9987    5 ...    0    0    0]
 [  27 9988    5 ...    0    0    0]
 [ 724  645    5 ...    0    0    0]]
[[   1   63    7 ...    0    0    0]
 [   1  360    3 ...    0    0    0]
 [   1  710    7 ...    0    0    0]
 ...
 [   1   27 9987 ...    0    0    0]
 [   1   27 9988 ...    0    0    0]
 [   1  724  645 ...    0    0    0]]


In [94]:
print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))

encoder_input_train=np.array(encoder_input[:30000], subok=True)
encoder_input_test=np.array(encoder_input[30000:], subok=True)
decoder_input_train=np.array(decoder_input[:30000], subok=True)
decoder_input_test=np.array(decoder_input[30000:], subok=True)
decoder_target_train=np.array(decoder_target[:30000], subok=True)
decoder_target_test=np.array(decoder_target[30000:], subok=True)
print(np.shape(encoder_input_train))
print(np.shape(encoder_input_test))

영어 데이터의 크기(shape) : (33000, 8)
프랑스어 입력데이터의 크기(shape) : (33000, 15)
프랑스어 출력데이터의 크기(shape) : (33000, 15)
(30000, 8)
(3000, 8)


In [96]:
#print(np.min(list(map(len, [x for x in decoder_output]))))
encoder_input_train

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)

In [ ]:
model.fit(x=[encoder_input_train, decoder_input_train], y=decoder_target_train, \
          validation_data = ([encoder_input_test, decoder_input_test], decoder_target_test)
          ,batch_size=128, epochs=50)
#

Epoch 1/50


ValueError: ignored

In [ ]:
type(decoder_input_train[2]),type(decoder_input_test[1]),type(decoder_target_train), type(decoder_target_train[2]),type(decoder_target_test), type(decoder_target_test[2])

(numpy.ndarray, numpy.ndarray, numpy.ndarray, list, numpy.ndarray, list)

In [ ]:
type(decoder_input),type(decoder_input[0])

(numpy.ndarray, numpy.ndarray)